In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.transforms import ToTensor
from PIL import Image
import os
import numpy as np
import pandas as pd

from functools import partial
from collections import OrderedDict

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [2]:
class Conv2dAuto(nn.Conv2d):
    # same as Conv2d but with padding
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs) # super() : inherit from baseclass
        self.padding =  (self.kernel_size[0] // 2, self.kernel_size[1] // 2) # dynamic add padding based on the kernel_size
        
conv3x3 = partial(Conv2dAuto, kernel_size=3, bias=False)

class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.in_channels, self.out_channels =  in_channels, out_channels
        self.blocks = nn.Identity()
        self.shortcut = nn.Identity()   
    
    def forward(self, x):
        residual = x
        if self.should_apply_shortcut:
            residual = self.shortcut(x)
        # print(residual.shape)
        x = self.blocks(x)
        # print(x.shape)
        x += residual # shapes of x and residual don't match when expansion > 1 and self.blocks = nn.Identity()
        return x
    
    @property # use method like attribute with "." syntax
    def should_apply_shortcut(self):
        return self.in_channels != self.out_channels
    
class ResNetResidualBlock(ResidualBlock):
    def __init__(self, in_channels, out_channels, expansion=1, downsampling=1, conv=conv3x3, *args, **kwargs):
        super().__init__(in_channels, out_channels)
        self.expansion, self.downsampling, self.conv = expansion, downsampling, conv
        self.shortcut = nn.Sequential(OrderedDict(
        {
            # Conv2d expects the input to be of shape [batch_size, input_channels, input_height, input_width]
            'conv' : nn.Conv2d(self.in_channels, self.expanded_channels, kernel_size=1,
                      stride=self.downsampling, bias=False),
            
            'bn' : nn.BatchNorm2d(self.expanded_channels)
        })) if self.should_apply_shortcut else None
        
        
    @property
    def expanded_channels(self):
        return self.out_channels * self.expansion
    
    @property
    def should_apply_shortcut(self):
        return self.in_channels != self.expanded_channels

In [3]:
# test ResidualBlock
block = ResidualBlock(1,2)
block(torch.tensor([1,1]))

tensor([2, 2])

In [4]:
# test ResNetResidualBlock ??
# x = torch.tensor([[[[1.0, 1.1]]]])
# block = ResNetResidualBlock(1,1, expansion=2)
# block(x)

### Add film

In [5]:
class mlp(nn.Module):
        def __init__(self, input_size, output_size):
            super(mlp, self).__init__()
            self.input_size = input_size
            self.output_size = output_size
            
            self.hidden_size = 30
            self.fc1 = torch.nn.Linear(self.input_size, self.hidden_size)
            self.fc2 = torch.nn.Linear(self.hidden_size,self.hidden_size)
            self.fc3 = torch.nn.Linear(self.hidden_size, self.output_size)
            
            self.relu = torch.nn.ReLU()
            self.sigmoid = torch.nn.Sigmoid() ## sigmoid for multi-label, softmax for multi-class (mutually exclusive)
            
        def forward(self, x):
            out = self.fc1(x)
            out = self.relu(out)
            
            out = self.fc2(out)
            out = self.relu(out)
            
            out = self.fc2(out)
            out = self.relu(out)
            
            out = self.fc2(out)
            out = self.relu(out)
            
            out = self.fc3(out)
            out = self.sigmoid(out)
            return out

In [6]:
class FiLM(nn.Module):
    def __init__(self, gamma, beta, *args, **kwargs):
        super().__init__(*args, **kwargs) # super() : inherit from baseclass
        self.gamma = gamma
        self.beta = beta
        
    def forward(self, x):
        # gammas = gammas.unsqueeze(2).unsqueeze(3).expand_as(x)
        # betas = betas.unsqueeze(2).unsqueeze(3).expand_as(x)
        gamma = self.gamma.expand_as(x)
        beta = self.beta.expand_as(x)
        
        return (gamma * x) + beta

In [7]:
def conv_bn(in_channels, out_channels, conv, *args, **kwargs):
    return nn.Sequential(OrderedDict({'conv': conv(in_channels, out_channels, *args, **kwargs), 
                                      'bn': nn.BatchNorm2d(out_channels)
                                      # 'film': FiLM(gamma, beta)
                                     }))

class ResNetBasicBlock(ResNetResidualBlock):
    expansion = 1
    def __init__(self, in_channels, out_channels, activation=nn.ReLU, *args, **kwargs):
        super().__init__(in_channels, out_channels, *args, **kwargs)
        self.blocks = nn.Sequential(
            # self.conv = conv3x3
            conv_bn(self.in_channels, self.out_channels, conv=self.conv, bias=False, stride=self.downsampling),
            activation(),
            conv_bn(self.out_channels, self.expanded_channels, conv=self.conv, bias=False),
        )

class ResNetBottleNeckBlock(ResNetResidualBlock):
    expansion = 4
    def __init__(self, in_channels, out_channels, activation=nn.ReLU, *args, **kwargs):
        super().__init__(in_channels, out_channels, expansion=4, *args, **kwargs)
        self.blocks = nn.Sequential(
           conv_bn(self.in_channels, self.out_channels, self.conv, film=self.film, kernel_size=1),
             activation(),
             conv_bn(self.out_channels, self.out_channels, self.conv, film=self.film, kernel_size=3, stride=self.downsampling),
             activation(),
             conv_bn(self.out_channels, self.expanded_channels, self.conv, film=self.film, kernel_size=1),
        )

### Finish building resnet

In [8]:
class ResNetLayer(nn.Module):
    # def __init__(self, in_channels, out_channels, gamma, beta, block=ResNetBasicBlock, n=1, *args, **kwargs):
    def __init__(self, in_channels, out_channels, block=ResNetBasicBlock, n=1, *args, **kwargs):
        super().__init__()
        # 'We perform downsampling directly by convolutional layers that have a stride of 2.'
        downsampling = 2 if in_channels != out_channels else 1
        
        self.blocks = nn.Sequential(
            block(in_channels , out_channels, *args, **kwargs, downsampling=downsampling),
            *[block(out_channels * block.expansion, 
                    out_channels, downsampling=1, *args, **kwargs) for _ in range(n - 1)]
        )
    
    # def forward(self, x, gamma, beta)
    def forward(self, x): #gamma beta
        x = self.blocks(x)
        # x = FiLM(x, gamma, beta)
        return x

class ResNetEncoder(nn.Module):
    """
    ResNet encoder composed by increasing different layers with increasing features.
    """
    def __init__(self, in_channels=3, blocks_sizes=[64, 128, 256, 512], deepths=[2,2,2,2], 
                 activation=nn.ReLU, block=ResNetBasicBlock, *args,**kwargs):
        super().__init__()
        
        self.blocks_sizes = blocks_sizes
        
        self.gate = nn.Sequential(
            nn.Conv2d(in_channels, self.blocks_sizes[0], kernel_size=7, stride=2, padding=3, bias=False),
            nn.BatchNorm2d(self.blocks_sizes[0]),
            activation(),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )
        
        self.in_out_block_sizes = list(zip(blocks_sizes, blocks_sizes[1:]))
        self.blocks = nn.ModuleList([ 
            ResNetLayer(blocks_sizes[0], blocks_sizes[0], n=deepths[0], activation=activation, 
                        block=block,  *args, **kwargs),
            *[ResNetLayer(in_channels * block.expansion, 
                          out_channels, n=n, activation=activation, 
                          block=block, *args, **kwargs) 
              for (in_channels, out_channels), n in zip(self.in_out_block_sizes, deepths[1:])]       
        ])
        
        # gammas[:, num_block, block_dim]
        
        
    def forward(self, x):
        x = self.gate(x)
        for block in self.blocks:
            x = block(x)
        return x

class ResnetDecoder(nn.Module):
    """
    This class represents the tail of ResNet. It performs a global pooling and maps the output to the
    correct class by using a fully connected layer.
    """
    def __init__(self, in_features, n_classes):
        super().__init__()
        self.avg = nn.AdaptiveAvgPool2d((1, 1))
        self.decoder = nn.Linear(in_features, n_classes)

    def forward(self, x):
        x = self.avg(x)
        x = x.view(x.size(0), -1)
        x = self.decoder(x)
        return x

class ResNet(nn.Module):
    
    def __init__(self, in_channels, n_classes, *args, **kwargs):
        super().__init__()
        self.encoder = ResNetEncoder(in_channels, *args, **kwargs)
        self.decoder = ResnetDecoder(self.encoder.blocks[-1].blocks[-1].expanded_channels, n_classes)
        # self.module_dim = 
        
    # def forward(self, x, film):
    def forward(self, x):
        # gammas, betas = torch.split(film[:,:,:2*self.module_dim], self.module_dim, dim=-1)
        
        x = self.encoder(x)
        x = self.decoder(x)
        return x

def resnet18(in_channels, n_classes):
    return ResNet(in_channels, n_classes, block=ResNetBasicBlock, deepths=[2, 2, 2, 2])

def resnet34(in_channels, n_classes):
    return ResNet(in_channels, n_classes, block=ResNetBasicBlock, deepths=[3, 4, 6, 3])

def resnet50(in_channels, n_classes):
    return ResNet(in_channels, n_classes, block=ResNetBottleNeckBlock, deepths=[3, 4, 6, 3])

def resnet101(in_channels, n_classes):
    return ResNet(in_channels, n_classes, block=ResNetBottleNeckBlock, deepths=[3, 4, 23, 3])

def resnet152(in_channels, n_classes):
    return ResNet(in_channels, n_classes, block=ResNetBottleNeckBlock, deepths=[3, 8, 36, 3])

## Dataset

In [9]:
df = pd.read_csv('data_stephen_fix_header.csv', header=[0])
# convert timecodes to year and month columns
datetimes = pd.to_datetime(df['time'])
df['month'] = datetimes.dt.month
df['year'] = datetimes.dt.year

print(df.shape[0])
print(df['borehole'].nunique())

2837
566


In [30]:
df.head()

,latitude,longitude,time,borehole,depth,frozen,cryostructures,visible_ice,ASTM_2488,materials,organic_cover,top_of_interval,bottom_of_interval,month,year
0,69.16162,-133.08682,2012-03-21T00:00:00Z,0170-1-10,0.15,0,NaN,NaN,TOPSOIL,Organics,0.3,0.0,0.3,3,2012
1,69.16162,-133.08682,2012-03-21T00:00:00Z,0170-1-10,0.85,1,NaN,Pure ice,ICE,Ice,0.3,0.3,1.4,3,2012
2,69.16162,-133.08682,2012-03-21T00:00:00Z,0170-1-10,1.90,1,Nf,No visible ice,SW-SM,Coarse till,0.3,1.4,2.4,3,2012
3,69.16162,-133.08682,2012-03-21T00:00:00Z,0170-1-10,5.40,1,Nf,No visible ice,GW-GM,Coarse till,0.3,2.4,8.4,3,2012
4,69.16105,-133.08880,2012-03-21T00:00:00Z,0170-1-12,1.20,1,Nf,No visible ice,GP-GM,Coarse till,0.0,0.0,2.4,3,2012


In [31]:
df['frozen'].sum()

2484

In [10]:
class Geo90Dataset(Dataset):
    def __init__(self, data_root, df, base_lat, base_lng, chip_size=20, label_name = 'frozen'):
        
        self.base_lat = base_lat
        self.base_lng = base_lng
        
        self.df = df
        
        self.chip_size = chip_size
        self.label_name = label_name
        
        self.trans = transforms.ToTensor()
        
        self.preloaded = torch.zeros(26, 6000, 6000)
        
        for i, file in enumerate(os.listdir("geomorph_data")):
            # name = file.split('_')[0]
            # print(name)
            self.preloaded[i] = self.trans(Image.open("geomorph_data/" + file))
        
    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        
        bh_id = row.at['borehole']
        lat = row.at['latitude']
        lng = row.at['longitude']
        

        pixel_len = 5/6000
        

        lat_index_start = np.round((self.base_lat - lat) / pixel_len - self.chip_size/2).astype(int)
        lat_index_end = lat_index_start + self.chip_size
        
        lng_index_start = np.round((lng - self.base_lng) / pixel_len - self.chip_size/2).astype(int)
        lng_index_end = lng_index_start + self.chip_size
        
        image = self.preloaded[:, lat_index_start:lat_index_end,lng_index_start:lng_index_end]
        
        
        surface = torch.tensor(row.filter(['latitude', 'longitude', 'year', 'month', 'depth']))
        
        return {'image': image, 'surface_data': surface, 'frozen': row.at['frozen']}


In [11]:
base_lat = 70
base_lng = -135

full_dataset = Geo90Dataset("geomorph_data", df, base_lat, base_lng, chip_size = 100)

## Train model

In [12]:
train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size

training_data, test_data = torch.utils.data.random_split(full_dataset, [train_size, test_size])

batchsize = 20

trainloader = DataLoader(training_data, batch_size=batchsize, shuffle=True)
testloader = DataLoader(test_data, batch_size=batchsize, shuffle=True)

In [13]:
film_gen = mlp()
gen_optimizer = torch.optim.Adam(film_gen.parameters())

film_net = resnet18(26, 2)
net_optimizer = torch.optim.Adam(film_net.parameters())

loss_fn = nn.CrossEntropyLoss()

for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0): # loop over each sample
        
        # get the inputs; data is a list of [inputs, labels]
        images, surface_data, labels = data['image'], data['surface_data'], data['frozen']
        
        # TODO: exammine film_params gradients / readup pytorch
        film_params = film_gen(surface_data)
        predicted = film_net(images, film_params)

        # predicted = film_net(images)
        loss = loss_fn(predicted, labels)

        gen_optimizer.zero_grad()
        net_optimizer.zero_grad()
        
        
        loss.backward()
        
        gen_optimizer.step()
        net_optimizer.step()
        
        # print statistics
        print_interval = 20
        running_loss += loss.item()
        if i % print_interval == print_interval-1:    # print every $print_interval mini-batches
            print('[%d, %5d] running loss: %.5f' %
                  (epoch + 1, i + 1, running_loss / print_interval))
            running_loss = 0.0

print('Finished Training')


c:\users\mouju\appdata\local\programs\python\python38\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


[1,    20] running loss: 1.12174
[1,    40] running loss: 0.51285
[1,    60] running loss: 0.40280
[1,    80] running loss: 0.38432
[1,   100] running loss: 0.40581
[2,    20] running loss: 0.43695
[2,    40] running loss: 0.38143
[2,    60] running loss: 0.33936
[2,    80] running loss: 0.46175
[2,   100] running loss: 0.37115
Finished Training


In [14]:
torch.save(film_net, 'film_net.pth')

## Test model

In [15]:
y_test = []
y_pred = []
for i, data in enumerate(testloader, 0):
    images, surface_data, label = data['image'], data['surface_data'], data['frozen']
    
    # y_test.append(label.numpy().list())
    # print(label.shape)
    # print(images.shape)
    outputs = film_net(images)
    _, predicted = torch.max(outputs, 1)
    # print(predicted.shape)
    lb = label.tolist()
    pd = predicted.tolist()
    y_test.extend(lb)
    y_pred.extend(pd)

In [16]:
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

[[  0  77]
 [  0 491]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        77
           1       0.86      1.00      0.93       491

    accuracy                           0.86       568
   macro avg       0.43      0.50      0.46       568
weighted avg       0.75      0.86      0.80       568

0.8644366197183099


c:\users\mouju\appdata\local\programs\python\python38\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\mouju\appdata\local\programs\python\python38\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\mouju\appdata\local\programs\python\python38\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

In [26]:
a = torch.arange(40).reshape(5,2,4)

In [27]:
b, c = torch.split(a, 2, -1)

In [28]:
b.shape

torch.Size([5, 2, 2])

In [20]:
c

tensor([[2, 3],
        [4, 5],
        [6, 7],
        [8, 9]])

In [29]:
blocks_sizes=[64, 128, 256, 512]
list(zip(blocks_sizes, blocks_sizes[1:]))

[(64, 128), (128, 256), (256, 512)]